In [1]:
print('123')

123


In [1]:
import pgl
from pgl import graph  # 导入 PGL 中的图模块
import paddle.fluid as fluid # 导入飞桨框架
import numpy as np

def build_graph():
    # 定义图中的节点数目，我们使用数字来表示图中的每个节点
    num_nodes = 10

    # 定义图中的边集
    edge_list = [(2, 0), (2, 1), (3, 1),(4, 0), (5, 0),
             (6, 0), (6, 4), (6, 5), (7, 0), (7, 1),
             (7, 2), (7, 3), (8, 0), (9, 7)]

    # 随机初始化节点特征，特征维度为 d
    d = 16
    feature = np.random.randn(num_nodes, d).astype("float32")

    # 随机地为每条边赋值一个权重
    edge_feature = np.random.randn(len(edge_list), 1).astype("float32")

    # 创建图对象，最多四个输入
    g = graph.Graph(num_nodes = num_nodes,
                    edges = edge_list,
                    node_feat = {'feature':feature},
                    edge_feat ={'edge_feature': edge_feature})

    return g

g = build_graph()

In [2]:
print('图中共计 %d 个节点' % g.num_nodes)
print('图中共计 %d 条边' % g.num_edges)

图中共计 10 个节点
图中共计 14 条边


In [3]:
# 定义一个同时传递节点特征和边权重的简单模型层。
def model_layer(gw, nfeat, efeat, hidden_size, name, activation):
    '''
    gw: GraphWrapper 图数据容器，用于在定义模型的时候使用，后续训练时再feed入真实数据
    nfeat: 节点特征
    efeat: 边权重
    hidden_size: 模型隐藏层维度
    activation: 使用的激活函数
    '''

    # 定义 send 函数
    def send_func(src_feat, dst_feat, edge_feat):
        # 将源节点的节点特征和边权重共同作为消息发送
        return src_feat['h'] * edge_feat['e']

    # 定义 recv 函数
    def recv_func(feat):
        # 目标节点接收源节点消息，采用 sum 的聚合方式
        return fluid.layers.sequence_pool(feat, pool_type='sum')

    # 触发消息传递机制
    msg = gw.send(send_func, nfeat_list=[('h', nfeat)], efeat_list=[('e', efeat)])
    output = gw.recv(msg, recv_func)
    output = fluid.layers.fc(output,
                    size=hidden_size,
                    bias_attr=False,
                    act=activation,
                    name=name)
    return output

In [4]:
class Model(object):
    def __init__(self, graph):
        """
        graph: 我们前面创建好的图
        """
        # 创建 GraphWrapper 图数据容器，用于在定义模型的时候使用，后续训练时再feed入真实数据
        self.gw = pgl.graph_wrapper.GraphWrapper(name='graph',
                    node_feat=graph.node_feat_info(),
                    edge_feat=graph.edge_feat_info())
        # 作用同 GraphWrapper，此处用作节点标签的容器
        self.node_label = fluid.layers.data("node_label", shape=[None, 1],
                    dtype="float32", append_batch_size=False)

    def build_model(self):
        # 定义两层model_layer
        output = model_layer(self.gw, 
                             self.gw.node_feat['feature'], 
                             self.gw.edge_feat['edge_feature'],
                             hidden_size=8, 
                             name='layer_1', 
                             activation='relu')
        output = model_layer(self.gw, 
                             output, 
                             self.gw.edge_feat['edge_feature'],
                             hidden_size=1, 
                             name='layer_2', 
                             activation=None)
                             
        # 对于二分类任务，可以使用以下 API 计算损失
        loss = fluid.layers.sigmoid_cross_entropy_with_logits(x=output, 
                                                              label=self.node_label)
        # 计算平均损失
        loss = fluid.layers.mean(loss)
        
        # 计算准确率
        prob = fluid.layers.sigmoid(output)
        pred = prob > 0.5
        pred = fluid.layers.cast(prob > 0.5, dtype="float32")
        correct = fluid.layers.equal(pred, self.node_label)
        correct = fluid.layers.cast(correct, dtype="float32")
        acc = fluid.layers.reduce_mean(correct)

        return loss, acc

In [5]:
# 是否在 GPU 或 CPU 环境运行
use_cuda = False
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()

# 定义程序，也就是我们的 Program
startup_program = fluid.Program() # 用于初始化模型参数
train_program = fluid.Program()   # 训练时使用的主程序，包含前向计算和反向梯度计算
test_program = fluid.Program()    # 测试时使用的程序，只包含前向计算

with fluid.program_guard(train_program, startup_program):
    model = Model(g)
    # 创建模型和计算 Loss
    loss, acc = model.build_model()
    # 选择Adam优化器，学习率设置为0.01
    adam = fluid.optimizer.Adam(learning_rate=0.01)
    adam.minimize(loss) # 计算梯度和执行梯度反向传播过程

# 复制构造 test_program，与 train_program的区别在于不需要梯度计算和反向过程。
test_program = train_program.clone(for_test=True)

# 定义一个在 place(CPU)上的Executor来执行program
exe = fluid.Executor(place)
# 参数初始化
exe.run(startup_program) 

# 获取真实图数据
feed_dict = model.gw.to_feed(g) 
# 获取真实标签数据
# 由于我们是做节点分类任务，因此可以简单的用0、1表示节点类别。其中，黄色点标签为0，绿色点标签为1。
y = [0,1,1,1,0,0,0,1,0,1]
label = np.array(y, dtype="float32")
label = np.expand_dims(label, -1)
feed_dict['node_label'] = label

/Users/zhengqixin/opt/anaconda3/envs/Paddle/lib/python3.6/site-packages/pgl/graph_wrapper.py:151: UserWarning: The edge features in argument `efeat_list` should be fetched from a instance of `pgl.graph_wrapper.GraphWrapper`, because we have sorted the edges and the order of edges is changed.
Therefore, if you use external edge features, the order of features of each edge may not match its edge, which can cause serious errors.
If you use the `efeat_list` correctly, please ignore this warning.
  "The edge features in argument `efeat_list` should be fetched "


In [7]:

for epoch in range(30):
    train_loss = exe.run(train_program,
        feed=feed_dict,      # feed入真实训练数据
        fetch_list=[loss],   # fetch出需要的计算结果
        return_numpy=True)[0]
    print('Epoch %d | Loss: %f' % (epoch, train_loss))

Epoch 0 | Loss: 0.590563
Epoch 1 | Loss: 0.544562
Epoch 2 | Loss: 0.510613
Epoch 3 | Loss: 0.486049
Epoch 4 | Loss: 0.467708
Epoch 5 | Loss: 0.454132
Epoch 6 | Loss: 0.444432
Epoch 7 | Loss: 0.437542
Epoch 8 | Loss: 0.432609
Epoch 9 | Loss: 0.429030
Epoch 10 | Loss: 0.426391
Epoch 11 | Loss: 0.424414
Epoch 12 | Loss: 0.422909
Epoch 13 | Loss: 0.421748
Epoch 14 | Loss: 0.420842
Epoch 15 | Loss: 0.420124
Epoch 16 | Loss: 0.419547
Epoch 17 | Loss: 0.419079
Epoch 18 | Loss: 0.418702
Epoch 19 | Loss: 0.418389
Epoch 20 | Loss: 0.418127
Epoch 21 | Loss: 0.417906
Epoch 22 | Loss: 0.417718
Epoch 23 | Loss: 0.417558
Epoch 24 | Loss: 0.417421
Epoch 25 | Loss: 0.417302
Epoch 26 | Loss: 0.417199
Epoch 27 | Loss: 0.417109
Epoch 28 | Loss: 0.417030
Epoch 29 | Loss: 0.416960
